# Analyze features for original and READII negative control CT images

Image features extracted from CT images cropped to a Gross Tumour Volume (GTV) undergo correlation analysis. Results are compared across READII negative control image types.

## Imports

In [1]:
import numpy as np
import pandas as pd
from damply import dirs
from readii.analyze.correlation import getFeatureCorrelations, getSelfCorrelations
from readii.io.loaders import loadFeatureFilesFromImageTypes, loadImageDatasetConfig
from readii.io.writers.correlation_writer import CorrelationWriter
from readii.process.config import get_full_data_name
from sklearn.cluster import AgglomerativeClustering


# from palettable.colorbrewer.diverging import PuOr_4

# import sys; sys.path.append("code")
# from analyze import prepPatientIndex, makeAllHeatmapPlots, makeAllHistogramPlots, makeAllClusterHeatmapPlots

/home/bhkuser1/katy/readii_2_roqc/.pixi/envs/default/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Initialize dataset name and load config 

In [2]:
dataset = "NSCLC-Radiomics_test"
dataset_config = loadImageDatasetConfig("NSCLC-Radiomics_test", dirs.CONFIG / "datasets")

DATASET_NAME = dataset_config['DATASET_NAME']
full_data_name = get_full_data_name(dirs.CONFIG / "datasets" / dataset)

readii_config = dataset_config['READII']

regions = readii_config['IMAGE_TYPES']['regions']

permutations = readii_config['IMAGE_TYPES']['permutations']

## Set up data directories

In [3]:
feature_type = dataset_config['EXTRACTION']['METHOD']
settings_name = dataset_config['EXTRACTION']['SETTINGS']

correlations_dir = dirs.PROCDATA / full_data_name / "correlations" / feature_type / settings_name

corr_figures_path = dirs.RESULTS / full_data_name / "correlation_figures" / feature_type / settings_name

# Set features input directory
features_dir = dirs.RESULTS / full_data_name / settings_name


## Load all extracted feature sets

In [4]:
# Load the extracted feature data
# This makes a dictionary of feature sets, with the image type as the keys
extracted_feature_sets = loadFeatureFilesFromImageTypes(extracted_feature_dir=features_dir,
                                                        image_types = (["original"] + negative_control_list), 
                                                        drop_labels = False)

# Run correlation analysis for each image type

Calculate the Pearson correlation between the features of each image type and itself (self-correlation) and between the original image and each negative control (cross-correlation).

Generates three figures to visualize the correlations:
1. *Heatmap* - self-correlations only show the bottom left half since it's symmetric, cross-correlations show a full heatmap
2. *Clustered heatmap* - original image feature correlations undergo hierarchical clustering with complete linkage, all feature sets are sorted by cluster and plotted as a heatmap. Same data as 1, just organized differently.
3. *Histogram* - visualize the distribution of Pearson correlation values for self and cross correlations

In [6]:
# Set up writers for correlation and plots
corr_matrix_writer = CorrelationWriter(
    root_directory = correlations_dir / "matrix",
    filename_format = DATASET_NAME + "_{VerticalFeatureType}_{HorizontalFeatureType}_{CorrelationType}_correlations.csv",
    overwrite = False,
    create_dirs = True
)

Note: if the kernel crashes when running the next cell, run the heatmap and clustered heatmap steps separately by commenting out one of them at a time

In [13]:
# Name of the column used to extract the patient ID for a row of features
file_path_column = 'image_path'

# Correlation method to apply
correlation_method = "pearson"

# Colormap to use for plots
heatmap_cmap = PuOr_4.mpl_colormap

# initialize clustering id array
clustering = np.array([])

# Whether to overwrite existing files
overwrite = False

# Get and set up the feature dataframe for the original features once
vertical_feature_type = "original"
vertical_features_df = prepPatientIndex(extracted_feature_sets[vertical_feature_type],
                                        file_path_column,
                                        PAT_ID_PATTERN)

# Iterate over each negative control feature set and perform correlation analysis
for horizontal_feature_type in negative_control_list:
    print(f"Processing {vertical_feature_type} vs {horizontal_feature_type} correlations.")

    # Generate the output path for matrix file existence check
    corr_matrix_output_path = corr_matrix_writer.resolve_path(VerticalFeatureType=vertical_feature_type,
                                                              HorizontalFeatureType=horizontal_feature_type,
                                                              CorrelationType=correlation_method)
    
    # Get extracted features for this image type, extract set the patient ID as the dataframe index, remove image_path column
    horizontal_features_df = prepPatientIndex(extracted_feature_sets[horizontal_feature_type], 
                                              file_path_column = file_path_column, 
                                              pat_id_pattern = PAT_ID_PATTERN)

    
    # Load existing correlation matrix if it's available
    if corr_matrix_output_path.exists() and corr_matrix_output_path.is_file():
        print("Loading correlation matrix.")
        feature_correlation_df = pd.read_csv(corr_matrix_output_path, index_col=0)
    
    # Calculate the correlation matrix if the file doesn't exist
    else:
        print("Calculating correlation matrix.")
        # Calculate correlations between original image features and image type features
        feature_correlation_df = getFeatureCorrelations(vertical_features=vertical_features_df,
                                                        horizontal_features=horizontal_features_df,
                                                        vertical_feature_name=vertical_feature_type,
                                                        horizontal_feature_name=horizontal_feature_type,
                                                        method = correlation_method)
        # save out the correlation dataframe
        corr_matrix_writer.save(feature_correlation_df, 
                                VerticalFeatureType=vertical_feature_type,
                                HorizontalFeatureType=horizontal_feature_type,
                                CorrelationType=correlation_method)
        
    print("Generating heatmaps for correlations.")
    vert_heatmap_path, horiz_heatmap_path, cross_heatmap_path = makeAllHeatmapPlots(feature_correlation_df,
                                                                                    vertical_feature_type,
                                                                                    horizontal_feature_type,
                                                                                    correlations_dir,
                                                                                    correlation_method,
                                                                                    heatmap_cmap,
                                                                                    overwrite)
    
    print("Generating heatmaps for clustered correlations.")
    if len(clustering) == 0:
        # Cluster the features based on the correlations from the Original image
        original_corr = getSelfCorrelations(feature_correlation_df, vertical_feature_type)
        clustering = AgglomerativeClustering(linkage="complete", metric="precomputed", n_clusters = None, distance_threshold = 0).fit_predict(original_corr)
    
    
    vert_heatmap_path, horiz_heatmap_path, cross_heatmap_path = makeAllClusterHeatmapPlots(feature_correlation_df,
                                                                                           vertical_feature_type,
                                                                                           horizontal_feature_type,
                                                                                           clustering,
                                                                                           correlations_dir / "clustered",
                                                                                           correlation_method,
                                                                                           heatmap_cmap,
                                                                                           overwrite)
    
    print("Generating histograms for correlations.")
    vert_histogram_path, horiz_histogram_path, cross_histogram_path = makeAllHistogramPlots(feature_correlation_df,
                                                                                            vertical_feature_type,
                                                                                            horizontal_feature_type,
                                                                                            correlations_dir,
                                                                                            correlation_method,
                                                                                            num_bins=450,
                                                                                            self_corr_y_max = 250000,
                                                                                            cross_corr_y_max = 950000,
                                                                                            overwrite=overwrite)
    


Processing original vs shuffled_full correlations.
Loading correlation matrix.
Generating heatmaps for correlations.
Plotting vertical feature correlations heatmap...
Plotting horizontal feature correlations heatmap...
Plotting cross feature correlations heatmap...
Generating heatmaps for clustered correlations.
Plotting vertical feature correlations heatmap...
Plotting horizontal feature correlations heatmap...
Plotting cross feature correlations heatmap...
Generating histograms for correlations.
Plotting vertical feature correlations histogram...


2025-04-11T11:39:55-0400 [warning  ] File ../../PublicDatasets/procdata/HeadNeck/TCIA_RADCURE/readii-fmcib/readii-fmcib_analysis/correlations/cropped_cube/histogram/original_pearson_self_correlation_histogram.png already exists. 
Set PlotWriter.overwrite to True to overwrite. [readii] call=plot_correlation.saveCorrelationHistogram:132


Plotting horizontal feature correlations histogram...


2025-04-11T11:39:56-0400 [warning  ] File ../../PublicDatasets/procdata/HeadNeck/TCIA_RADCURE/readii-fmcib/readii-fmcib_analysis/correlations/cropped_cube/histogram/shuffled_full_pearson_self_correlation_histogram.png already exists. 
Set PlotWriter.overwrite to True to overwrite. [readii] call=plot_correlation.saveCorrelationHistogram:132


Plotting cross feature correlations histogram...


2025-04-11T11:39:57-0400 [warning  ] File ../../PublicDatasets/procdata/HeadNeck/TCIA_RADCURE/readii-fmcib/readii-fmcib_analysis/correlations/cropped_cube/histogram/original_vs_shuffled_full_pearson_cross_correlation_histogram.png already exists. 
Set PlotWriter.overwrite to True to overwrite. [readii] call=plot_correlation.saveCorrelationHistogram:132


Processing original vs shuffled_roi correlations.
Loading correlation matrix.
Generating heatmaps for correlations.
Plotting vertical feature correlations heatmap...


2025-04-11T11:40:25-0400 [warning  ] File ../../PublicDatasets/procdata/HeadNeck/TCIA_RADCURE/readii-fmcib/readii-fmcib_analysis/correlations/cropped_cube/heatmap/<matplotlib.colors.LinearSegmentedColormap object at 0x7f2368f41e20>/original_pearson_self_correlation_heatmap.png already exists. 
Set PlotWriter.overwrite to True to overwrite. [readii] call=plot_correlation.saveCorrelationHeatmap:73


Plotting horizontal feature correlations heatmap...
Plotting cross feature correlations heatmap...
Generating heatmaps for clustered correlations.
Plotting vertical feature correlations heatmap...


: 

# Original Correlations Only
Run this section if no negative controls are being run and you want to analyze just the original image features.

In [ ]:
from readii.analyze.plot_correlation import plotSelfCorrHeatmap, plotSelfCorrHistogram

# Name of the column used to extract the patient ID for a row of features
file_path_column = 'image_path'

# Correlation method to apply
correlation_method = "pearson"

# Colormap to use for plots
heatmap_cmap = "nipy_spectral"

# Whether to overwrite existing files
overwrite = True

# Get and set up the feature dataframe for the original features once
vertical_feature_type = "original"
vertical_features_df = prepPatientIndex(extracted_feature_sets[vertical_feature_type],
                                        file_path_column,
                                        PAT_ID_PATTERN)

# Generate the output path for matrix file existence check
corr_matrix_output_path = corr_matrix_writer.resolve_path(VerticalFeatureType=vertical_feature_type,
                                                          HorizontalFeatureType=vertical_feature_type,
                                                          CorrelationType=correlation_method)

# Load existing correlation matrix if it's available
if corr_matrix_output_path.exists() and corr_matrix_output_path.is_file():
    print("Loading correlation matrix.")
    vertical_self_corr_df = pd.read_csv(corr_matrix_output_path, index_col=0)

# Calculate the correlation matrix if the file doesn't exist
else:
    print("Calculating correlation matrix.")
    # Calculate correlations between vertical image features 
    vertical_self_corr_df = vertical_features_df.corr(method = correlation_method)
    # Add feature type suffix to column names and index for the plotting functions to work
    vertical_self_corr_df = vertical_self_corr_df.add_suffix(f"_{vertical_feature_type}", 0)
    vertical_self_corr_df = vertical_self_corr_df.add_suffix(f"_{vertical_feature_type}", 1)

    # save out the correlation dataframe
    corr_matrix_writer.save(vertical_self_corr_df, 
                            VerticalFeatureType=vertical_feature_type,
                            HorizontalFeatureType=vertical_feature_type,
                            CorrelationType=correlation_method)
    
# Make plots
_, vert_heatmap_path = plotSelfCorrHeatmap(vertical_self_corr_df,
                                           vertical_feature_type,
                                           correlation_method,
                                           heatmap_cmap,
                                           correlations_dir,
                                           overwrite)
    
_, vert_histogram_path = plotSelfCorrHistogram(vertical_self_corr_df,
                                               vertical_feature_type,
                                               correlation_method,
                                               num_bins=450,
                                               y_upper_bound = 100000,
                                               save_dir_path=correlations_dir,
                                               overwrite=overwrite)